In [1]:
import numpy as np
from transformers import pipeline
from transformers import AutoProcessor, BarkModel
import scipy

# Audio-to-Text

In [3]:
asr = pipeline("automatic-speech-recognition", model="openai/whisper-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [4]:
result = asr("/content/03LAB3.wav")
print(result['text'])

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:604: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


 Физик проверил ввод.


# Классификация аудио

классификация по полу

In [ ]:
classifier = pipeline("audio-classification", model="alefiury/wav2vec2-large-xlsr-53-gender-recognition-librispeech")
result = classifier("/content/03LAB3.wav")
print(result)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

Device set to use cpu


[{'score': 0.9985658526420593, 'label': 'female'}, {'score': 0.0014341691276058555, 'label': 'male'}]


классификация по эмоциям

In [2]:
classifier2 = pipeline("audio-classification", model="firdhokk/speech-emotion-recognition-with-openai-whisper-large-v3")
result2 = classifier2("/content/03LAB3.wav")
print(result2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.55G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

Device set to use cuda:0


[{'score': 0.5844483971595764, 'label': 'happy'}, {'score': 0.3688162863254547, 'label': 'surprised'}, {'score': 0.03903479129076004, 'label': 'neutral'}, {'score': 0.003601806703954935, 'label': 'angry'}, {'score': 0.0021175756119191647, 'label': 'sad'}, {'score': 0.0013590484159067273, 'label': 'fearful'}, {'score': 0.0006220248760655522, 'label': 'disgust'}]


# Text-to-Speech

In [3]:
processor = AutoProcessor.from_pretrained("suno/bark")
model = BarkModel.from_pretrained("suno/bark")

prompt = "Привет! Это пример синтеза речи через модель Bark."
inputs = processor(prompt, return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

speaker_embeddings_path.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.49G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

In [5]:
audio_array = model.generate(**inputs)
audio = audio_array[0] if len(audio_array.shape) > 1 else audio_array
scaled_audio = np.int16(audio.numpy() * 32767)
scipy.io.wavfile.write("output.wav", rate=22050, data=scaled_audio)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


Озвучка не плохая, но на фоне какой-то шум. В конце интонация как будто планируется продолжение речи.

# Text-to-Audio

Вместо прямой генерации аудиофайла, она создаёт изображение спектрограммы, а затем преобразует его в звук.

In [6]:

# Импорты
import torch
from diffusers import DiffusionPipeline
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from scipy.io.wavfile import write
from PIL import Image

# Модель Riffusion
pipe = DiffusionPipeline.from_pretrained("riffusion/riffusion-model-v1", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

# Текстовый промпт
prompt = "ambient synth melody with soft bass"

# Генерация спектрограммы
image = pipe(prompt).images[0]

# Отображение спектрограммы (если нужно)
plt.imshow(image)
plt.axis('off')
plt.show()

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--riffusion--riffusion-model-v1/snapshots/8f2e752c74e8316c6eb4fdaa6598a46ce1d88af5/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--riffusion--riffusion-model-v1/snapshots/8f2e752c74e8316c6eb4fdaa6598a46ce1d88af5/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--riffusion--riffusion-model-v1/snapshots/8f2e752c74e8316c6eb4fdaa6598a46ce1d88af5/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--riffusion--riffusion-model-v1/snapshots/8f2e752c74e8316c6eb4fdaa6598a46ce1d88af5/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
/usr/local/lib/python3.11/dist-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffu

  0%|          | 0/50 [00:00<?, ?it/s]

In [8]:
# Преобразование изображения спектрограммы в аудио
def image_to_audio(image: Image.Image, output_path: str = "output_rif.wav", sr: int = 22050):
    img = image.resize((512, 512)).convert("L")  # градации серого
    img_array = np.array(img).astype(np.float32) / 255.0
    img_array = img_array[::-1]  # инвертировать по вертикали

    # Восстановление аудио через Griffin-Lim
    S = librosa.db_to_amplitude(img_array * 80 - 80)  # масштабирование в дБ
    audio = librosa.griffinlim(S)

    # Сохранение в WAV
    scaled = np.int16(audio / np.max(np.abs(audio)) * 32767)
    write(output_path, sr, scaled)
    print(f"Аудио сохранено как {output_path}")

# Генерация и сохранение WAV-файла
image_to_audio(image)

Аудио сохранено как output_rif.wav


мелодия частично подходит под описание
